# 과제: 네이버 영화 정보 및 평점 크롤링

- 대상: 예매순 상위 5개의 현재 상영 중인 영화
- 수집할 항목: 영화 제목, 주연배우 3인, 네티즌 평점, 관람객 평점, 기자/평론가 평점, 관람객 별점 리뷰 20건 공감순으로(평점, 작성자닉네임, 리뷰본문)

★ 이 파일은 모듈화 안돼있는거 테스트하는 용도로 만들었음

##### 가이드라인
1. 아래 가이드라인은 채점 방식과 무관 과제하기 편하라고 임의로 만든 틀입니다. 다 지우고 본인 편한 방식대로 해도 됩니다!
2. 채점할 때 경로 문제가 발생하지 않도록 경로는 전부 상대경로로 지정해 주세요. (경로 이름에 직접적으로 C: D: 등이 들어가지 않음)

##### 멘토링 가이드라인
1. 답지는 번호를 임의로 나눠둔 부분에 대해서 함수를 만든 거라서 더 효율적인 방법이 있으면 그렇게 해도 됨
2. 강의 때 안 배운 함수들이 답지에 있어서 찾아보면서 해야 함
3. 정적 크롤링 방식으로 했을 때 바로 안 뜨는 부분은 selenium으로 해도 되겠지만 웬만하면 네트워크에서 받아와서 하는 방식으로

[네트워크에서 별점 리뷰 찾기] https://www.yceffort.kr/2018/11/05/web-crwaling-for-naver-movie/
> https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=186821&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2

4. n 번째 태그 찾는 방법 https://stackoverflow.com/questions/37149955/find-third-occurring-p-tag-using-with-beautiful-soup
5. 정답은 json 파일로 저장하는 걸로 만들었지만 저장 방식은 자유~~!
6. 그리고 크롤링 방법이 정말 다양하기 때문에 정답이 뭐라고 알려주기보다 이런 방식도 있다고 알려주면 좋을듯

.

In [47]:
import requests
from bs4 import BeautifulSoup
import re

### 1. 예매순 상위 5개의 현재 상영 중인 영화 가져오기

영화 제목, 주연배우 3인

In [3]:
url = 'https://movie.naver.com/movie/running/current.nhn'

In [4]:
res = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
html = res.text

In [5]:
soup = BeautifulSoup(html, 'html.parser')

In [25]:
titles = soup.find_all(class_ ='tit')
for title in titles[0:5]:
    print(title.find('a').text)    
    print(title.find('a').get('href'))

정직한 후보
/movie/bi/mi/basic.nhn?code=186821
작은 아씨들
/movie/bi/mi/basic.nhn?code=186613
클로젯
/movie/bi/mi/basic.nhn?code=181925
기생충
/movie/bi/mi/basic.nhn?code=161967
수퍼 소닉
/movie/bi/mi/basic.nhn?code=172816


In [70]:
infos = soup.find_all(class_ ='info_txt1')
for info in infos[0:5]:
    actors_list = info.select_one("dd:nth-of-type(3)").text.strip().split(',')
    actors = []
    n = 1
    for actor in actors_list:
        if n==4:
            break
        print(actor.strip())
        n+=1

라미란
김무열
나문희
시얼샤 로넌
엠마 왓슨
플로렌스 퓨
하정우
김남길
허율
송강호
이선균
조여정
짐 캐리
제임스 마스던
벤 슈와츠


### 2. 해당 영화의 평점 가져오기

네티즌 평점, 관람객 평점, 기자/평론가 평점

In [71]:
address = '/movie/bi/mi/basic.nhn?code=186821'
url = 'https://movie.naver.com'+address

In [82]:
res = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
html = res.text

In [83]:
soup = BeautifulSoup(html, 'html.parser')

In [136]:
p = re.compile('[0-9]{2}.[0-9]{1}')

In [137]:
likey = soup.find_all(class_ ='st_on')

In [141]:
for like in likey[0:3]:
    print(float(p.findall(like.get('style'))[0])*0.1)

9.14
5.38
7.96


### 3. 관람객 평점 공감순 20건 가져오기

평점, 평점 작성자 닉네임, 리뷰 본문

In [143]:
url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page}'.format(code=186821, page=1)

In [144]:
res = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
html = res.text

In [145]:
soup = BeautifulSoup(html, 'html.parser')

In [163]:
score_result = soup.find('div', {'class': 'score_result'})
lis = score_result.findAll('li')
lis[0]

<li>
<div class="star_score">
<span class="st_off"><span class="st_on" style="width:100.0%"></span></span><em>10</em>
</div>
<div class="score_reple">
<p>
<span class="ico_viewer">관람객</span>
<!-- 스포일러 컨텐츠로 처리되는지 여부 -->
<span id="_filtered_ment_0">
														
															
															
																너무 재밌었어용ㅠ 라미란 짱멋… 
															
														
														
													</span>
</p>
<dl>
<dt>
<em>
<a href="#" onclick="javascript:showPointListByNid(16703919, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>김다솜(kmjd****)</span>
</a>
</em>
<em>2020.02.12 17:13</em>
</dt>
<dd>
<a class="go_report2" href="#" onclick="parent.clickcr(this, 'ara.report', '', '', event); common.report('false','kmjd****', 'rO9vfZ1PtXZ9oGjeSw0ncGyezfFWblnS8ArF32eaIAI=', '너무 재밌었어용ㅠ 라미란 짱멋… ', '16703919', 'point_after', false);return false;"><em>신고</em></a>
</dd>
</dl>
</div>
<div class="btn_area">
<a class="_sympathyButton" href="#" onclick="parent.clickc

In [176]:
review_text = lis[0].find('p').text
review_text[4:].strip()

'너무 재밌었어용ㅠ 라미란 짱멋…'

In [166]:
lis[0].find('em').text

'10'

In [169]:
nickname = lis[0].findAll('a')[0].find('span').getText()
nickname

'김다솜(kmjd****)'